In [ ]:
import numpy as np
import csv
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
import pandas as pd

'''
    This demonstrates how to reach a 0.80 ROC AUC score (local 4-fold validation)
    in the Kaggle Nile virus prediction challenge. 

    The model trains in a few seconds on CPU.
'''

# let's define some utils

def get_weather_data():
    weather_dic = {}
    fi = csv.reader(open("asset/weather.csv"))
    weather_head = fi.__next__()
    for line in fi:
        if line[0] == '1':
            continue
        weather_dic[line[1]] = line
    weather_indexes = dict([(weather_head[i], i) for i in range(len(weather_head))])
    return weather_dic, weather_indexes

def process_line(line, indexes, weather_dic, weather_indexes):
    def get(name):
        return line[indexes[name]]

    date = get("Date")
    month = float(date.split('-')[1])
    week = int(date.split('-')[1]) * 4 + int(date.split('-')[2]) / 7
    latitude = float(get("Latitude"))
    longitude = float(get("Longitude"))
    tmax = float(weather_dic[date][weather_indexes["Tmax"]])
    tmin = float(weather_dic[date][weather_indexes["Tmin"]])
    tavg = float(weather_dic[date][weather_indexes["Tavg"]])
    dewpoint = float(weather_dic[date][weather_indexes["DewPoint"]])
    wetbulb = float(weather_dic[date][weather_indexes["WetBulb"]])
    pressure = float(weather_dic[date][weather_indexes["StnPressure"]])

    return [month, week, latitude, longitude, tmax, tmin, tavg, dewpoint, wetbulb, pressure]

def preprocess_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler

def shuffle(X, y, seed=1337):
    np.random.seed(seed)
    shuffle = np.arange(len(y))
    np.random.shuffle(shuffle)
    X = X[shuffle]
    y = y[shuffle]
    return X, y

def build_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(output_dim))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adadelta")
    return model


# now the actual script

print("Processing training data...")

rows = []
labels = []
fi = csv.reader(open("asset/train.csv"))
head = fi.__next__()
indexes = dict([(head[i], i) for i in range(len(head))])
weather_dic, weather_indexes = get_weather_data()
for line in fi:
    rows.append(process_line(line, indexes, weather_dic, weather_indexes))
    labels.append(float(line[indexes["WnvPresent"]]))

    
train = pd.read_csv('train_w_weather.csv')
test = pd.read_csv('test_w_weather.csv')

X = train.values
y = np.array(labels)

X, y = shuffle(X, y)
X, scaler = preprocess_data(X)
Y = np_utils.to_categorical(y)

input_dim = X.shape[1]
output_dim = 2

print("Validation...")

nb_folds = 4
kfolds = KFold(len(y), nb_folds)
av_roc = 0.
f = 0
for train, valid in kfolds:
    print('---'*20)
    print('Fold', f)
    print('---'*20)
    f += 1
    X_train = X[train]
    X_valid = X[valid]
    Y_train = Y[train]
    Y_valid = Y[valid]
    y_valid = y[valid]

    print("Building model...")
    model = build_model(input_dim, output_dim)

    print("Training model...")

    model.fit(X_train, Y_train, nb_epoch=100, batch_size=16, validation_data=(X_valid, Y_valid), verbose=0)
    valid_preds = model.predict_proba(X_valid, verbose=0)
    valid_preds = valid_preds[:, 1]
    roc = metrics.roc_auc_score(y_valid, valid_preds)
    print("ROC:", roc)
    av_roc += roc

print('Average ROC:', av_roc/nb_folds)

print("Generating submission...")

# model = build_model(input_dim, output_dim)
# model.fit(X, Y, nb_epoch=100, batch_size=16, verbose=0)

# fi = csv.reader(open("asset/test.csv"))
# head = fi.__next__()
# indexes = dict([(head[i], i) for i in range(len(head))])
# rows = []
# ids = []
# for line in fi:
#     rows.append(process_line(line, indexes, weather_dic, weather_indexes))
#     ids.append(line[0])
# X_test = np.array(rows)
# X_test, _ = preprocess_data(X_test, scaler)

# preds = model.predict_proba(X_test, verbose=0)

# fo = csv.writer(open("keras-nn.csv", "w"), lineterminator="\n")
# fo.writerow(["Id","WnvPresent"])

# for i, item in enumerate(ids):
#     fo.writerow([ids[i], preds[i][1]])

Processing training data...
Validation...
------------------------------------------------------------
Fold 0
------------------------------------------------------------
Building model...
Training model...


/home/roland/anaconda3/lib/python3.6/site-packages/keras/models.py:846: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ROC: 0.76632403873
------------------------------------------------------------
Fold 1
------------------------------------------------------------
Building model...
Training model...


In [7]:
fi = csv.reader(open("asset/test.csv"))
head = fi.__next__()
indexes = dict([(head[i], i) for i in range(len(head))])
rows = []
ids = []
for line in fi:
    rows.append(process_line(line, indexes, weather_dic, weather_indexes))
    ids.append(line[0])
X_test = test.values
X_test, _ = preprocess_data(X_test, scaler)

preds = model.predict_proba(X_test, verbose=0)

fo = csv.writer(open("keras-nn.csv", "w"), lineterminator="\n")
fo.writerow(["Id","WnvPresent"])

for i, item in enumerate(ids):
    fo.writerow([ids[i], preds[i][1]])

In [28]:
X_test = test.values
X_test, _ = preprocess_data(X_test, scaler)

preds = model.predict_proba(X_test, verbose=0)
sample = pd.read_csv('asset/sampleSubmission.csv')
sample['WnvPresent'] = preds[:,1]
sample.to_csv('beat_the_benchmark_nn.csv', index=False)

In [29]:
sample

Id    WnvPresent
0            1  3.213933e-23
1            2  3.215563e-24
2            3  4.049486e-23
3            4  1.028233e-26
4            5  2.249674e-26
5            6  3.886310e-30
6            7  4.101035e-23
7            8  6.072100e-35
8            9  1.830061e-26
9           10  1.762296e-27
10          11  2.385357e-26
11          12  5.525751e-30
12          13  1.209147e-29
13          14  2.088545e-33
14          15  2.381393e-26
15          16  3.720002e-38
16          17  2.284250e-24
17          18  2.285442e-25
18          19  2.877929e-24
19          20  8.086128e-28
20          21  1.781248e-27
21          22  2.925236e-31
22          23  2.914520e-24
23          24  4.315644e-36
24          25  6.826195e-25
25          26  6.573430e-26
26          27  8.897333e-25
27          28  2.061126e-28
28          29  4.509888e-28
29          30  7.791421e-32
...        ...           ...
116263  116264  1.847332e-08
116264  116265  2.130639e-11
116265  116266  4.203410e-11
116266  116267  8.635420e-15
116267  116268  1.807543e-08
116268  116269  3.346561e-19
116269  116270  2.623652e-01
116270  116271  2.593169e-01
116271  116272  2.624275e-01
116272  116273  2.480766e-01
116273  116274  2.490009e-01
116274  116275  9.225848e-02
116275  116276  2.627837e-01
116276  116277  2.832737e-04
116277  116278  1.542080e-08
116278  116279  2.759802e-09
116279  116280  2.058114e-08
116280  116281  2.332528e-11
116281  116282  5.070899e-11
116282  116283  1.436111e-14
116283  116284  1.935013e-08
116284  116285  4.652767e-19
116285  116286  3.167693e-02
116286  116287  1.506114e-02
116287  116288  3.295425e-02
116288  116289  5.097670e-04
116289  116290  9.657817e-04
116290  116291  3.706815e-07
116291  116292  3.320811e-02
116292  116293  1.655676e-11

[116293 rows x 2 columns]

In [22]:
np.shape(pd.read_csv("beat_the_benchmark_nn.csv"))

(116293, 2)